<a href="https://colab.research.google.com/github/MN-21/handwriting-dnn-features/blob/main/Sobel_Gradient_pipeline_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install --quiet tensorflow tensorflow-datasets opencv-python tqdm scikit-learn

import cv2
import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. Sobel feature extractor (full-image gradients)
def compute_gradients(images):
    """
    Given a NumPy array of shape (N,28,28), returns
    an array of shape (N, 2, 28, 28) containing normalized gx, gy.
    """
    N = images.shape[0]
    grads = np.zeros((N, 2, 28, 28), dtype=np.float32)
    for i, img in enumerate(tqdm(images, desc="Computing gradients")):
        gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
        gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
        # normalize each to [0,1]
        gx = (gx - gx.min()) / (gx.max() - gx.min() + 1e-8)
        gy = (gy - gy.min()) / (gy.max() - gy.min() + 1e-8)
        grads[i,0] = gx
        grads[i,1] = gy
    return grads

# 2. Load MNIST via TFDS
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

# 3. Convert to NumPy arrays
all_images = []
all_labels = []
for img, lbl in tfds.as_numpy(ds_train.concatenate(ds_test)):
    all_images.append(img.squeeze())  # shape (28,28)
    all_labels.append(int(lbl))       # labels 0–9
all_images = np.stack(all_images)    # (70000,28,28)
all_labels = np.array(all_labels, dtype=int)

# 4. Stratified 80/20 train/test split
X_train, X_test, y_train, y_test = train_test_split(
    all_images, all_labels,
    train_size=0.8,
    stratify=all_labels,
    random_state=42
)

# 5. Compute Sobel gradient features
grad_train = compute_gradients(X_train)
grad_test  = compute_gradients(X_test)

# 6. Flatten to feature vectors (2*28*28 = 1568 dims)
X_train_feat = grad_train.reshape(-1, 2*28*28)
X_test_feat  = grad_test.reshape(-1, 2*28*28)

# 7. Build MLP model for 10 classes
model = models.Sequential([
    layers.Input(shape=(2*28*28,)),
    layers.Dense(1024), layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.5),

    layers.Dense(512), layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.4),

    layers.Dense(256), layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.3),

    layers.Dense(10, activation='softmax')  # 10 output classes for MNIST digits
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 8. Callbacks
es = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=4,
    restore_best_weights=True, verbose=1
)
rlr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5,
    patience=3, min_lr=1e-6, verbose=1
)

# 9. Train
t0 = time.time()
history = model.fit(
    X_train_feat, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=128,
    callbacks=[es, rlr],
    verbose=2
)
train_time = time.time() - t0

# 10. Evaluate
t1 = time.time()
test_loss, test_acc = model.evaluate(X_test_feat, y_test, verbose=2)
infer_time = time.time() - t1

# 11. Report
print(f"\nMNIST Sobel-Gradient MLP")
print(f"Test Accuracy   : {test_acc*100:.2f}%")
print(f"Training Time   : {train_time:.1f}s")
print(f"Inference Time  : {infer_time:.1f}s for {X_test_feat.shape[0]} samples")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.PQA7HF_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Computing gradients: 100%|██████████| 14000/14000 [00:00<00:00, 18679.81it/s]


Epoch 1/50
394/394 - 19s - 48ms/step - accuracy: 0.8886 - loss: 0.3662 - val_accuracy: 0.5509 - val_loss: 2.1708 - learning_rate: 1.0000e-03
Epoch 2/50
394/394 - 15s - 39ms/step - accuracy: 0.9469 - loss: 0.1760 - val_accuracy: 0.6477 - val_loss: 1.8559 - learning_rate: 1.0000e-03
Epoch 3/50
394/394 - 16s - 40ms/step - accuracy: 0.9587 - loss: 0.1338 - val_accuracy: 0.5071 - val_loss: 2.3769 - learning_rate: 1.0000e-03
Epoch 4/50
394/394 - 21s - 53ms/step - accuracy: 0.9650 - loss: 0.1127 - val_accuracy: 0.6488 - val_loss: 1.5042 - learning_rate: 1.0000e-03
Epoch 5/50
394/394 - 16s - 41ms/step - accuracy: 0.9691 - loss: 0.0982 - val_accuracy: 0.2082 - val_loss: 6.5478 - learning_rate: 1.0000e-03
Epoch 6/50
394/394 - 16s - 40ms/step - accuracy: 0.9727 - loss: 0.0865 - val_accuracy: 0.6273 - val_loss: 1.7502 - learning_rate: 1.0000e-03
Epoch 7/50

Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
394/394 - 16s - 41ms/step - accuracy: 0.9754 - loss: 0.0746 - val_